In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.4 MB/s 
     |████████████████████████████████| 182 kB 59.1 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 


In [4]:
model_name="bert-base-uncased"

In [5]:
from transformers import AutoModelForSequenceClassification
new_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/bert-base-uncased/model")

In [6]:
from transformers import AutoTokenizer
new_tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import nltk
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
def preprocessing(text):
    ps = PorterStemmer()
    sentence = re.sub('[^a-zA-Z]',' ',text) # noktalama işaretlerini silme.
    sentence = sentence.lower() # kelimelerin hepsini küçük harf 
    sentence = sentence.split() # keliemeleri listeye atma
    sentence = [ps.stem(kelime) for kelime in sentence if not kelime in set(stopwords.words("english"))] #anlamı olmayan(stopwords) kelimeleri listeden atma
    sentence = " ".join(sentence) # listede kalan kelimeleri birleştirip yeni cümleyi oluşturma.
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
sentence = preprocessing("Do you want to go school?")
print(sentence)

want go school


In [9]:
import torch
import numpy as np

def get_prediction(text):
    text = preprocessing(text)
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    # encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = new_model(**encoding)

    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1) # değeri 0 veya 1'e yuvarlıyoruz.
    
    if label == 1:
        print(f"sentiment: True, probability: {probs[1]}")
        return 1
    else:
        print(f"sentiment: Fake, probability: {probs[1]}")
        return 0

In [10]:
get_prediction("He goes to airport.")

sentiment: True, probability: 0.9462123513221741


1

In [11]:
submission = pd.read_csv("/content/drive/MyDrive/submission.csv")
# y = submission["Truth"]
# x = submission["Haber"]

In [12]:
accuracy = 0
y_pred = []
for i in range(submission.shape[0]):
    row = submission.iloc[i]
    text = row[0]
    truth_value = row[1]
    print(f"{i}-)")
    result = get_prediction(text)
    y_pred.append(result)
    if(result == int(truth_value)):
        accuracy += 1 
accuracy

0-)
sentiment: True, probability: 0.9462123513221741
1-)
sentiment: Fake, probability: 0.04909549281001091
2-)
sentiment: Fake, probability: 0.04909117519855499
3-)
sentiment: True, probability: 0.9462127089500427
4-)
sentiment: Fake, probability: 0.0490933358669281
5-)
sentiment: True, probability: 0.9462121725082397
6-)
sentiment: True, probability: 0.9462121725082397
7-)
sentiment: Fake, probability: 0.04910113662481308
8-)
sentiment: True, probability: 0.9462123513221741
9-)
sentiment: True, probability: 0.9462122917175293
10-)
sentiment: True, probability: 0.9462123513221741
11-)
sentiment: True, probability: 0.9462122917175293
12-)
sentiment: True, probability: 0.9462123513221741
13-)
sentiment: True, probability: 0.9462130069732666
14-)
sentiment: True, probability: 0.9462121725082397
15-)
sentiment: True, probability: 0.9462121725082397
16-)
sentiment: Fake, probability: 0.04910892993211746
17-)
sentiment: True, probability: 0.9462123513221741
18-)
sentiment: Fake, probability:

970

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
y_true=submission["Truth"].tolist()

In [15]:
conf_matrix=confusion_matrix(y_true, y_pred)
conf_matrix

array([[472,  20],
       [ 10, 498]])

In [16]:
TP = conf_matrix[1][1]
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
print('True Positives:', TP)
print('True Negatives:', TN)
print('False Positives:', FP)
print('False Negatives:', FN)

True Positives: 498
True Negatives: 472
False Positives: 20
False Negatives: 10


In [17]:
conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))

conf_error_rate = 1- conf_accuracy
    
conf_sensitivity = (TP / float(TP + FN)) #recall

conf_specificity = (TN / float(TN + FP))
    
conf_precision = (TN / float(TN + FP))

conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
print('-'*50)
print(f'Accuracy: {round(conf_accuracy,2)}') 
print(f'Error_rate: {round(conf_error_rate,2)}') 
print(f'Sensitivity: {round(conf_sensitivity,2)}') 
print(f'Specificity: {round(conf_specificity,2)}') 
print(f'Precision: {round(conf_precision,2)}')
print(f'f_1 Score: {round(conf_f1,2)}')

--------------------------------------------------
Accuracy: 0.97
Error_rate: 0.03
Sensitivity: 0.98
Specificity: 0.96
Precision: 0.96
f_1 Score: 0.97
